In [8]:
import pickle
import requests
import pandas as pd

# Cargamos el modelo entrenado
with open('modelo_entrenado_v1.pk1', 'rb') as f:
    model = pickle.load(f)

In [32]:
# Llave privada de YELP API
API_KEY = 'ZCiemO3dscUrsLb8INf1d3KaA6hZPEkhbmvCu6va162wz4c2G-04aiCGEux729ZetfonkA41i68V2AIsFMxhXR68sa0xteZQRmot_J8KhRjvU_KFRxVxurlgOuJGZHYx'
headers = {'Authorization': f'Bearer {API_KEY}'}
params = {'term': 'McDonalds', 'location': 'New York'}

url = 'https://api.yelp.com/v3/businesses/search'
response = requests.get(url, headers=headers, params=params)

businesses = response.json()['businesses']

business_ids = []

if businesses:
    x = 0
    for i in businesses:
        a = i['id']
        business_ids.append(a)
        x += 1
else:
    print("No businesses found for the search term.")
    
business_ids

['0l-l2j5NAIqtychRNs17Dw',
 'LR6nRVRoP0g_wqqlIE8Ynw',
 'PGbKlytumI2WaIefhtCALw',
 'B_HkncKcyD0ZL3LsHmnzqg',
 'Um1WimKhVLw3KB-OzRYfyQ',
 'YV0VaJgV2zotzI7XNUaMCQ',
 'oXVZ9OqrC55fp-3hDBvRCQ',
 'jPIZ3FR5LNcwPuUHi2Fe4g',
 'EW7kFuXTsXRQ4Kp7j6qxHw',
 '0Ao9422D82WgsiUbr_t_Qg',
 'in7QM4vNUTHVZIi7Outetg',
 'NBVvD__J9JjomKtdds6Gvg',
 'cLXIlxKwnvUPg8JoxdfNwg',
 'hcVN7B1tw4jvRTiORTFKjA',
 '_yInwn1jLMDuPaSuap-Jrg',
 'Tr942SPajn138V7ZDx7Ckw',
 'AiOb7w3Z9jS8XJdamgbU_A',
 'J8n4z6pqHt8tiHkAaA0JVg',
 'RbhDdQfSY4x_CM4HQDSHUA',
 'p4EQbVuLddcTrgw_K8n8OA']

In [35]:

# set up parameters for the Yelp API request
url2= 'https://api.yelp.com/v3/businesses/{restaurant_id}/reviews'
params2 = {'sort_by': 'desc'}

# make the API request
response = requests.get(url2.format(restaurant_id='0l-l2j5NAIqtychRNs17Dw'), headers=headers, params=params2)
reviews = response.json()['reviews']

# create a pandas DataFrame from the review data
df = pd.DataFrame(reviews, columns=['id', 'text', 'rating', 'time_created', 'user'])

# display the DataFrame
df

,id,text,rating,time_created,user
0,A_T8F4Na1-5oVWvHrOs6_Q,1)east 14th st location. my friend Carol who s...,2,2023-03-23 08:24:41,"{'id': 'DLefWraiObJrlmmeT1hQ7Q', 'profile_url'..."
1,3gbFjayzRE3QMwhlYololA,Pretty decent McDonalds to get decently priced...,3,2023-03-31 16:45:34,"{'id': 'ibPNMYiIAZf51AWhgmRRBQ', 'profile_url'..."
2,lHw1G0aLfj8hVlS6QQqOpQ,The woman behind the counter was very rude to ...,1,2023-04-20 07:11:32,"{'id': '__Mg1kdxhl1N3GLx0al-ow', 'profile_url'..."
